# Data Drift and Model Performance Dashboards for Iris dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets, model_selection, neighbors

from evidently.dashboard import Dashboard
from evidently.tabs import DriftTab, CatTargetDriftTab, ClassificationPerformanceTab

## Iris data

In [2]:
iris = datasets.load_iris()

In [3]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

## Data drift report 

In [4]:
iris_data_drift_report = Dashboard(iris_frame, iris_frame, column_mapping = None, tabs=[DriftTab])

In [5]:
iris_data_drift_report.show()

In [6]:
iris_data_drift_report.save('reports/iris_data_drift.html')

## Data and Target drift reports 

In [7]:
iris_frame['target'] = iris.target

In [8]:
iris_data_and_target_drift_report = Dashboard(iris_frame[:75], iris_frame[75:], 
                                              column_mapping = None, tabs=[DriftTab, CatTargetDriftTab])

/Users/emeli/Dev/evidently/dev/lib/python3.6/site-packages/scipy/stats/stats.py:5982: RuntimeWarning:

divide by zero encountered in true_divide



In [9]:
iris_data_and_target_drift_report.show()

In [10]:
iris_data_and_target_drift_report.save('reports/iris_data_and_target_drift.html')

## Model Performance report

In [11]:
iris_frame = pd.DataFrame(iris.data, columns = iris.feature_names)

In [12]:
reference, production, y_train, y_test = model_selection.train_test_split(iris_frame, 
                                                                          iris.target, 
                                                                          random_state=0)

In [13]:
model = neighbors.KNeighborsClassifier(n_neighbors=1)

In [14]:
model.fit(reference, y_train)

KNeighborsClassifier(n_neighbors=1)

In [15]:
train_predictions = model.predict(reference)
test_predictions = model.predict(production)

In [16]:
reference['target'] = y_train
reference['prediction'] = train_predictions

production['target'] = y_test
production['prediction'] = test_predictions

In [17]:
reference.target = reference.target.apply(lambda x: iris.target_names[x])
reference.prediction = reference.prediction.apply(lambda x: iris.target_names[x])

production.target = production.target.apply(lambda x: iris.target_names[x])
production.prediction = production.prediction.apply(lambda x: iris.target_names[x])

In [18]:
iris_column_mapping = {}

iris_column_mapping['target'] = 'target'
iris_column_mapping['prediction'] = 'prediction'
iris_column_mapping['numerical_features'] = iris.feature_names

In [19]:
iris_drift = Dashboard(reference, production, column_mapping = iris_column_mapping, 
                       tabs=[ClassificationPerformanceTab])
iris_drift.show()

In [20]:
iris_drift.save('reports/iris_classification_performance.html')